In [1]:
import pandas as pd
import numpy as np
import re
import regex

from tqdm import tqdm
tqdm.pandas()

# Reading cells 

We have manually made tables in `etc/`, we just annotate them with POS and concatenate to make a single table:

In [2]:
LatInfLexi_verbs_cells = pd.read_csv("etc/LatInfLexi-verbs_cells.csv")
LatInfLexi_verbs_cells["POS"] = "verb"
LatInfLexi_nouns_cells = pd.read_csv("etc/LatInfLexi-nouns_cells.csv")
LatInfLexi_nouns_cells["POS"] = "noun"
LatInfLexi_cells = pd.concat([LatInfLexi_verbs_cells, LatInfLexi_nouns_cells])

# Reading forms

We get the previous version's forms

In [3]:
%%bash 
git checkout v1.1 -- LatInfLexi-nouns.csv
git checkout v1.1 -- LatInfLexi-verbs.csv

We read these tables

In [4]:
LatInfLexi_verbs = pd.read_csv("LatInfLexi-verbs.csv")
LatInfLexi_nouns = pd.read_csv("LatInfLexi-nouns.csv")

# Making some corrections

In [6]:
changes = {
# vowel length in the present system of lexeme "praescisco" (and consequently IPA transcriptions) 
633730: { "form" : "praescīscō" , "form_IPA" : "prajskiːskoː" } ,
633731: { "form" : "praescīscis" , "form_IPA" : "prajskiːskis" } ,
633732: { "form" : "praescīscit" , "form_IPA" : "prajskiːskit" } ,
633733: { "form" : "praescīscimus" , "form_IPA" : "prajskiːskimus" } ,
633734: { "form" : "praescīscitis" , "form_IPA" : "prajskiːskitis" } ,
633735: { "form" : "praescīscunt" , "form_IPA" : "prajskiːskunt" } ,
633736: { "form" : "praescīscēbam" , "form_IPA" : "prajskiːskeːbam" } ,
633737: { "form" : "praescīscēbās" , "form_IPA" : "prajskiːskeːbaːs" } ,
633738: { "form" : "praescīscēbat" , "form_IPA" : "prajskiːskeːbat" } ,
633739: { "form" : "praescīscēbāmus" , "form_IPA" : "prajskiːskeːbaːmus" } ,
633740: { "form" : "praescīscēbātis" , "form_IPA" : "prajskiːskeːbaːtis" } ,
633741: { "form" : "praescīscēbant" , "form_IPA" : "prajskiːskeːbant" } ,
633742: { "form" : "praescīscam" , "form_IPA" : "prajskiːskam" } ,
633743: { "form" : "praescīscēs" , "form_IPA" : "prajskiːskeːs" } ,
633744: { "form" : "praescīscet" , "form_IPA" : "prajskiːsket" } ,
633745: { "form" : "praescīscēmus" , "form_IPA" : "prajskiːskeːmus" } ,
633746: { "form" : "praescīscētis" , "form_IPA" : "prajskiːskeːtis" } ,
633747: { "form" : "praescīscent" , "form_IPA" : "prajskiːskent" } ,
633748: { "form" : "praescīscam" , "form_IPA" : "prajskiːskam" } ,
633749: { "form" : "praescīscās" , "form_IPA" : "prajskiːskaːs" } ,
633750: { "form" : "praescīscat" , "form_IPA" : "prajskiːskat" } ,
633751: { "form" : "praescīscāmus" , "form_IPA" : "prajskiːskaːmus" } ,
633752: { "form" : "praescīscātis" , "form_IPA" : "prajskiːskaːtis" } ,
633753: { "form" : "praescīscant" , "form_IPA" : "prajskiːskant" } ,
633754: { "form" : "praescīscerem" , "form_IPA" : "prajskiːskerem" } ,
633755: { "form" : "praescīscerēs" , "form_IPA" : "prajskiːskereːs" } ,
633756: { "form" : "praescīsceret" , "form_IPA" : "prajskiːskeret" } ,
633757: { "form" : "praescīscerēmus" , "form_IPA" : "prajskiːskereːmus" } ,
633758: { "form" : "praescīscerētis" , "form_IPA" : "prajskiːskereːtis" } ,
633759: { "form" : "praescīscerent" , "form_IPA" : "prajskiːskerent" } ,
633760: { "form" : "praescīsce" , "form_IPA" : "prajskiːske" } ,
633761: { "form" : "praescīscite" , "form_IPA" : "prajskiːskite" } ,
633762: { "form" : "praescīscitō" , "form_IPA" : "prajskiːskitoː" } ,
633763: { "form" : "praescīscitō" , "form_IPA" : "prajskiːskitoː" } ,
633764: { "form" : "praescīscitōte" , "form_IPA" : "prajskiːskitoːte" } ,
633765: { "form" : "praescīscuntō" , "form_IPA" : "prajskiːskuntoː" } ,
633766: { "form" : "praescīscere" , "form_IPA" : "prajskiːskere" } ,
633767: { "form" : "praescīscor" , "form_IPA" : "prajskiːskor" } ,
633768: { "form" : "praescīsceris" , "form_IPA" : "prajskiːskeris" } ,
633769: { "form" : "praescīscitur" , "form_IPA" : "prajskiːskitur" } ,
633770: { "form" : "praescīscimur" , "form_IPA" : "prajskiːskimur" } ,
633771: { "form" : "praescīsciminī" , "form_IPA" : "prajskiːskiminiː" } ,
633772: { "form" : "praescīscuntur" , "form_IPA" : "prajskiːskuntur" } ,
633773: { "form" : "praescīscēbar" , "form_IPA" : "prajskiːskeːbar" } ,
633774: { "form" : "praescīscēbāris" , "form_IPA" : "prajskiːskeːbaːris" } ,
633775: { "form" : "praescīscēbātur" , "form_IPA" : "prajskiːskeːbaːtur" } ,
633776: { "form" : "praescīscēbāmur" , "form_IPA" : "prajskiːskeːbaːmur" } ,
633777: { "form" : "praescīscēbāminī" , "form_IPA" : "prajskiːskeːbaːminiː" } ,
633778: { "form" : "praescīscēbantur" , "form_IPA" : "prajskiːskeːbantur" } ,
633779: { "form" : "praescīscar" , "form_IPA" : "prajskiːskar" } ,
633780: { "form" : "praescīscēris" , "form_IPA" : "prajskiːskeːris" } ,
633781: { "form" : "praescīscētur" , "form_IPA" : "prajskiːskeːtur" } ,
633782: { "form" : "praescīscēmur" , "form_IPA" : "prajskiːskeːmur" } ,
633783: { "form" : "praescīscēminī" , "form_IPA" : "prajskiːskeːminiː" } ,
633784: { "form" : "praescīscentur" , "form_IPA" : "prajskiːskentur" } ,
633785: { "form" : "praescīscar" , "form_IPA" : "prajskiːskar" } ,
633786: { "form" : "praescīscāris" , "form_IPA" : "prajskiːskaːris" } ,
633787: { "form" : "praescīscātur" , "form_IPA" : "prajskiːskaːtur" } ,
633788: { "form" : "praescīscāmur" , "form_IPA" : "prajskiːskaːmur" } ,
633789: { "form" : "praescīscāminī" , "form_IPA" : "prajskiːskaːminiː" } ,
633790: { "form" : "praescīscantur" , "form_IPA" : "prajskiːskantur" } ,
633791: { "form" : "praescīscerer" , "form_IPA" : "prajskiːskerer" } ,
633792: { "form" : "praescīscerēris" , "form_IPA" : "prajskiːskereːris" } ,
633793: { "form" : "praescīscerētur" , "form_IPA" : "prajskiːskereːtur" } ,
633794: { "form" : "praescīscerēmur" , "form_IPA" : "prajskiːskereːmur" } ,
633795: { "form" : "praescīscerēminī" , "form_IPA" : "prajskiːskereːminiː" } ,
633796: { "form" : "praescīscerentur" , "form_IPA" : "prajskiːskerentur" } ,
633797: { "form" : "praescīscere" , "form_IPA" : "prajskiːskere" } ,
633798: { "form" : "praescīsciminī" , "form_IPA" : "prajskiːskiminiː" } ,
633799: { "form" : "praescīscitor" , "form_IPA" : "prajskiːskitor" } ,
633800: { "form" : "praescīscitor" , "form_IPA" : "prajskiːskitor" } ,
633801: { "form" : "praescīscuntor" , "form_IPA" : "prajskiːskuntor" } ,
633802: { "form" : "praescīscī" , "form_IPA" : "prajskiːskiː" } ,
633803: { "form" : "praescīscendī" , "form_IPA" : "prajskiːskendiː" } ,
633804: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633805: { "form" : "praescīscendum" , "form_IPA" : "prajskiːskendum" } ,
633806: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633807: { "form" : "praescīscendus" , "form_IPA" : "prajskiːskendus" } ,
633808: { "form" : "praescīscenda" , "form_IPA" : "prajskiːskenda" } ,
633809: { "form" : "praescīscendum" , "form_IPA" : "prajskiːskendum" } ,
633810: { "form" : "praescīscendī" , "form_IPA" : "prajskiːskendiː" } ,
633811: { "form" : "praescīscendae" , "form_IPA" : "prajskiːskendaj" } ,
633812: { "form" : "praescīscendī" , "form_IPA" : "prajskiːskendiː" } ,
633813: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633814: { "form" : "praescīscendae" , "form_IPA" : "prajskiːskendaj" } ,
633815: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633816: { "form" : "praescīscendum" , "form_IPA" : "prajskiːskendum" } ,
633817: { "form" : "praescīscendam" , "form_IPA" : "prajskiːskendam" } ,
633818: { "form" : "praescīscendum" , "form_IPA" : "prajskiːskendum" } ,
633819: { "form" : "praescīscende" , "form_IPA" : "prajskiːskende" } ,
633820: { "form" : "praescīscenda" , "form_IPA" : "prajskiːskenda" } ,
633821: { "form" : "praescīscendum" , "form_IPA" : "prajskiːskendum" } ,
633822: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633823: { "form" : "praescīscendā" , "form_IPA" : "prajskiːskendaː" } ,
633824: { "form" : "praescīscendō" , "form_IPA" : "prajskiːskendoː" } ,
633825: { "form" : "praescīscendī" , "form_IPA" : "prajskiːskendiː" } ,
633826: { "form" : "praescīscendae" , "form_IPA" : "prajskiːskendaj" } ,
633827: { "form" : "praescīscenda" , "form_IPA" : "prajskiːskenda" } ,
633828: { "form" : "praescīscendōrum" , "form_IPA" : "prajskiːskendoːrum" } ,
633829: { "form" : "praescīscendārum" , "form_IPA" : "prajskiːskendaːrum" } ,
633830: { "form" : "praescīscendōrum" , "form_IPA" : "prajskiːskendoːrum" } ,
633831: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633832: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633833: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633834: { "form" : "praescīscendōs" , "form_IPA" : "prajskiːskendoːs" } ,
633835: { "form" : "praescīscendās" , "form_IPA" : "prajskiːskendaːs" } ,
633836: { "form" : "praescīscenda" , "form_IPA" : "prajskiːskenda" } ,
633837: { "form" : "praescīscendī" , "form_IPA" : "prajskiːskendiː" } ,
633838: { "form" : "praescīscendae" , "form_IPA" : "prajskiːskendaj" } ,
633839: { "form" : "praescīscenda" , "form_IPA" : "prajskiːskenda" } ,
633840: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633841: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633842: { "form" : "praescīscendīs" , "form_IPA" : "prajskiːskendiːs" } ,
633843: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633844: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633845: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633846: { "form" : "praescīscentis" , "form_IPA" : "prajskiːskentis" } ,
633847: { "form" : "praescīscentis" , "form_IPA" : "prajskiːskentis" } ,
633848: { "form" : "praescīscentis" , "form_IPA" : "prajskiːskentis" } ,
633849: { "form" : "praescīscentī" , "form_IPA" : "prajskiːskentiː" } ,
633850: { "form" : "praescīscentī" , "form_IPA" : "prajskiːskentiː" } ,
633851: { "form" : "praescīscentī" , "form_IPA" : "prajskiːskentiː" } ,
633852: { "form" : "praescīscentem" , "form_IPA" : "prajskiːskentem" } ,
633853: { "form" : "praescīscentem" , "form_IPA" : "prajskiːskentem" } ,
633854: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633855: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633856: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633857: { "form" : "praescīscēns" , "form_IPA" : "prajskiːskeːns" } ,
633858: { "form" : "praescīscente" , "form_IPA" : "prajskiːskente" } ,
633859: { "form" : "praescīscente" , "form_IPA" : "prajskiːskente" } ,
633860: { "form" : "praescīscente" , "form_IPA" : "prajskiːskente" } ,
633861: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633862: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633863: { "form" : "praescīscentia" , "form_IPA" : "prajskiːskentia" } ,
633864: { "form" : "praescīscentium" , "form_IPA" : "prajskiːskentium" } ,
633865: { "form" : "praescīscentium" , "form_IPA" : "prajskiːskentium" } ,
633866: { "form" : "praescīscentium" , "form_IPA" : "prajskiːskentium" } ,
633867: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
633868: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
633869: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
633870: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633871: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633872: { "form" : "praescīscentia" , "form_IPA" : "prajskiːskentia" } ,
633873: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633874: { "form" : "praescīscentēs" , "form_IPA" : "prajskiːskenteːs" } ,
633875: { "form" : "praescīscentia" , "form_IPA" : "prajskiːskentia" } ,
633876: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
633877: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
633878: { "form" : "praescīscentibus" , "form_IPA" : "prajskiːskentibus" } ,
# "i" in prs.act.ind.2/3sg of lexeme "aio" is vowel, not glide (cf. Bennett grammar, §135)
41149: { "form_IPA" : "aiːs" } ,
41150: { "form_IPA" : "ait" } ,
# recovering passive forms of transitive derivatives of irregular "eo" (cf. Bennett grammar, §132.1, identification of transitives based on lemlat's codles
1857: { "form" : "adeor" , "form_ipa" : "adeor" } ,
1858: { "form" : "adīris" , "form_ipa" : "adiːris" } ,
1858: { "form" : "adīmur" , "form_ipa" : "adiːmur" } ,
1858: { "form" : "adīminī" , "form_ipa" : "adiːminiː" } ,
1858: { "form" : "adeuntur" , "form_ipa" : "adeuntur" } ,
1858: { "form" : "adībar" , "form_ipa" : "adiːbar" } ,
1858: { "form" : "adībāris" , "form_ipa" : "adiːbaːris" } ,
1858: { "form" : "adībāmur" , "form_ipa" : "adiːbaːmur" } ,
1858: { "form" : "adībāminī" , "form_ipa" : "adiːbaːminiː" } ,
1859: { "form" : "adībantur" , "form_ipa" : "adiːbantur" } ,
1859: { "form" : "adībor" , "form_ipa" : "adiːbor" } ,
1859: { "form" : "adīberis" , "form_ipa" : "adiːberis" } ,
1859: { "form" : "adībimur" , "form_ipa" : "adiːbimur" } ,
1859: { "form" : "adībiminī" , "form_ipa" : "adiːbiminiː" } ,
1859: { "form" : "adībuntur" , "form_ipa" : "adiːbuntur" } ,
1859: { "form" : "adear" , "form_ipa" : "adear" } ,
1859: { "form" : "adeāris" , "form_ipa" : "adeaːris" } ,
1860: { "form" : "adeāmur" , "form_ipa" : "adeaːmur" } ,
1860: { "form" : "adeāminī" , "form_ipa" : "adeaːminiː" } ,
1860: { "form" : "adeantur" , "form_ipa" : "adeantur" } ,
1860: { "form" : "adīrer" , "form_ipa" : "adiːrer" } ,
1860: { "form" : "adīrēris" , "form_ipa" : "adiːreːris" } ,
1860: { "form" : "adīrēmur" , "form_ipa" : "adiːreːmur" } ,
1860: { "form" : "adīrēminī" , "form_ipa" : "adiːreːminiː" } ,
1860: { "form" : "adīrentur" , "form_ipa" : "adiːrentur" } ,
1860: { "form" : "adīre" , "form_ipa" : "adiːre" } ,
1861: { "form" : "adīminī" , "form_ipa" : "adiːminiː" } ,
1861: { "form" : "adītor" , "form_ipa" : "adiːtor" } ,
1861: { "form" : "adītor" , "form_ipa" : "adiːtor" } ,
1861: { "form" : "adeuntor" , "form_ipa" : "adeuntor" } ,
1861: { "form" : "adeundus" , "form_ipa" : "adeundus" } ,
1862: { "form" : "adeunda" , "form_ipa" : "adeunda" } ,
1862: { "form" : "adeundum" , "form_ipa" : "adeundum" } ,
1862: { "form" : "adeundī" , "form_ipa" : "adeundiː" } ,
1862: { "form" : "adeundae" , "form_ipa" : "adeundaj" } ,
1862: { "form" : "adeundī" , "form_ipa" : "adeundiː" } ,
1862: { "form" : "adeundō" , "form_ipa" : "adeundoː" } ,
1862: { "form" : "adeundae" , "form_ipa" : "adeundaj" } ,
1862: { "form" : "adeundō" , "form_ipa" : "adeundoː" } ,
1862: { "form" : "adeundum" , "form_ipa" : "adeundum" } ,
1862: { "form" : "adeundam" , "form_ipa" : "adeundam" } ,
1863: { "form" : "adeunde" , "form_ipa" : "adeunde" } ,
1863: { "form" : "adeunda" , "form_ipa" : "adeunda" } ,
1863: { "form" : "adeundum" , "form_ipa" : "adeundum" } ,
1863: { "form" : "adeundō" , "form_ipa" : "adeundoː" } ,
1863: { "form" : "adeundā" , "form_ipa" : "adeundaː" } ,
1863: { "form" : "adeundō" , "form_ipa" : "adeundoː" } ,
1863: { "form" : "adeundī" , "form_ipa" : "adeundiː" } ,
1863: { "form" : "adeundae" , "form_ipa" : "adeundaj" } ,
1863: { "form" : "adeunda" , "form_ipa" : "adeunda" } ,
1864: { "form" : "adeundōrum" , "form_ipa" : "adeundoːrum" } ,
1864: { "form" : "adeundārum" , "form_ipa" : "adeundaːrum" } ,
1864: { "form" : "adeundōrum" , "form_ipa" : "adeundoːrum" } ,
1864: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
1864: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
1864: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
1864: { "form" : "adeundōs" , "form_ipa" : "adeundoːs" } ,
1864: { "form" : "adeundās" , "form_ipa" : "adeundaːs" } ,
1864: { "form" : "adeunda" , "form_ipa" : "adeunda" } ,
1864: { "form" : "adeundī" , "form_ipa" : "adeundiː" } ,
1865: { "form" : "adeundae" , "form_ipa" : "adeundaj" } ,
1865: { "form" : "adeunda" , "form_ipa" : "adeunda" } ,
1865: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
1865: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
1865: { "form" : "adeundīs" , "form_ipa" : "adeundiːs" } ,
5337: { "form" : "anteeor" , "form_ipa" : "anteeor" } ,
5337: { "form" : "anteīris" , "form_ipa" : "anteiːris" } ,
5338: { "form" : "anteīmur" , "form_ipa" : "anteiːmur" } ,
5338: { "form" : "anteīminī" , "form_ipa" : "anteiːminiː" } ,
5338: { "form" : "anteeuntur" , "form_ipa" : "anteeuntur" } ,
5338: { "form" : "anteībar" , "form_ipa" : "anteiːbar" } ,
5338: { "form" : "anteībāris" , "form_ipa" : "anteiːbaːris" } ,
5338: { "form" : "anteībāmur" , "form_ipa" : "anteiːbaːmur" } ,
5338: { "form" : "anteībāminī" , "form_ipa" : "anteiːbaːminiː" } ,
5338: { "form" : "anteībantur" , "form_ipa" : "anteiːbantur" } ,
5338: { "form" : "anteībor" , "form_ipa" : "anteiːbor" } ,
5339: { "form" : "anteīberis" , "form_ipa" : "anteiːberis" } ,
5339: { "form" : "anteībimur" , "form_ipa" : "anteiːbimur" } ,
5339: { "form" : "anteībiminī" , "form_ipa" : "anteiːbiminiː" } ,
5339: { "form" : "anteībuntur" , "form_ipa" : "anteiːbuntur" } ,
5339: { "form" : "anteear" , "form_ipa" : "anteear" } ,
5339: { "form" : "anteeāris" , "form_ipa" : "anteeaːris" } ,
5339: { "form" : "anteeāmur" , "form_ipa" : "anteeaːmur" } ,
5339: { "form" : "anteeāminī" , "form_ipa" : "anteeaːminiː" } ,
5340: { "form" : "anteeantur" , "form_ipa" : "anteeantur" } ,
5340: { "form" : "anteīrer" , "form_ipa" : "anteiːrer" } ,
5340: { "form" : "anteīrēris" , "form_ipa" : "anteiːreːris" } ,
5340: { "form" : "anteīrēmur" , "form_ipa" : "anteiːreːmur" } ,
5340: { "form" : "anteīrēminī" , "form_ipa" : "anteiːreːminiː" } ,
5340: { "form" : "anteīrentur" , "form_ipa" : "anteiːrentur" } ,
5340: { "form" : "anteīre" , "form_ipa" : "anteiːre" } ,
5340: { "form" : "anteīminī" , "form_ipa" : "anteiːminiː" } ,
5340: { "form" : "anteītor" , "form_ipa" : "anteiːtor" } ,
5341: { "form" : "anteītor" , "form_ipa" : "anteiːtor" } ,
5341: { "form" : "anteeuntor" , "form_ipa" : "anteeuntor" } ,
5341: { "form" : "anteeundus" , "form_ipa" : "anteeundus" } ,
5341: { "form" : "anteeunda" , "form_ipa" : "anteeunda" } ,
5341: { "form" : "anteeundum" , "form_ipa" : "anteeundum" } ,
5342: { "form" : "anteeundī" , "form_ipa" : "anteeundiː" } ,
5342: { "form" : "anteeundae" , "form_ipa" : "anteeundaj" } ,
5342: { "form" : "anteeundī" , "form_ipa" : "anteeundiː" } ,
5342: { "form" : "anteeundō" , "form_ipa" : "anteeundoː" } ,
5342: { "form" : "anteeundae" , "form_ipa" : "anteeundaj" } ,
5342: { "form" : "anteeundō" , "form_ipa" : "anteeundoː" } ,
5342: { "form" : "anteeundum" , "form_ipa" : "anteeundum" } ,
5342: { "form" : "anteeundam" , "form_ipa" : "anteeundam" } ,
5342: { "form" : "anteeunde" , "form_ipa" : "anteeunde" } ,
5343: { "form" : "anteeunda" , "form_ipa" : "anteeunda" } ,
5343: { "form" : "anteeundum" , "form_ipa" : "anteeundum" } ,
5343: { "form" : "anteeundō" , "form_ipa" : "anteeundoː" } ,
5343: { "form" : "anteeundā" , "form_ipa" : "anteeundaː" } ,
5343: { "form" : "anteeundō" , "form_ipa" : "anteeundoː" } ,
5343: { "form" : "anteeundī" , "form_ipa" : "anteeundiː" } ,
5343: { "form" : "anteeundae" , "form_ipa" : "anteeundaj" } ,
5343: { "form" : "anteeunda" , "form_ipa" : "anteeunda" } ,
5343: { "form" : "anteeundōrum" , "form_ipa" : "anteeundoːrum" } ,
5343: { "form" : "anteeundārum" , "form_ipa" : "anteeundaːrum" } ,
5344: { "form" : "anteeundōrum" , "form_ipa" : "anteeundoːrum" } ,
5344: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
5344: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
5344: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
5344: { "form" : "anteeundōs" , "form_ipa" : "anteeundoːs" } ,
5344: { "form" : "anteeundās" , "form_ipa" : "anteeundaːs" } ,
5344: { "form" : "anteeunda" , "form_ipa" : "anteeunda" } ,
5344: { "form" : "anteeundī" , "form_ipa" : "anteeundiː" } ,
5344: { "form" : "anteeundae" , "form_ipa" : "anteeundaj" } ,
5344: { "form" : "anteeunda" , "form_ipa" : "anteeunda" } ,
5345: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
5345: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
5345: { "form" : "anteeundīs" , "form_ipa" : "anteeundiːs" } ,
10112: { "form" : "circumeor" , "form_ipa" : "kirkumeor" } ,
10113: { "form" : "circumīris" , "form_ipa" : "kirkumiːris" } ,
10113: { "form" : "circumīmur" , "form_ipa" : "kirkumiːmur" } ,
10113: { "form" : "circumīminī" , "form_ipa" : "kirkumiːminiː" } ,
10113: { "form" : "circumeuntur" , "form_ipa" : "kirkumeuntur" } ,
10113: { "form" : "circumībar" , "form_ipa" : "kirkumiːbar" } ,
10113: { "form" : "circumībāris" , "form_ipa" : "kirkumiːbaːris" } ,
10113: { "form" : "circumībāmur" , "form_ipa" : "kirkumiːbaːmur" } ,
10113: { "form" : "circumībāminī" , "form_ipa" : "kirkumiːbaːminiː" } ,
10114: { "form" : "circumībantur" , "form_ipa" : "kirkumiːbantur" } ,
10114: { "form" : "circumībor" , "form_ipa" : "kirkumiːbor" } ,
10114: { "form" : "circumīberis" , "form_ipa" : "kirkumiːberis" } ,
10114: { "form" : "circumībimur" , "form_ipa" : "kirkumiːbimur" } ,
10114: { "form" : "circumībiminī" , "form_ipa" : "kirkumiːbiminiː" } ,
10114: { "form" : "circumībuntur" , "form_ipa" : "kirkumiːbuntur" } ,
10114: { "form" : "circumear" , "form_ipa" : "kirkumear" } ,
10114: { "form" : "circumeāris" , "form_ipa" : "kirkumeaːris" } ,
10115: { "form" : "circumeāmur" , "form_ipa" : "kirkumeaːmur" } ,
10115: { "form" : "circumeāminī" , "form_ipa" : "kirkumeaːminiː" } ,
10115: { "form" : "circumeantur" , "form_ipa" : "kirkumeantur" } ,
10115: { "form" : "circumīrer" , "form_ipa" : "kirkumiːrer" } ,
10115: { "form" : "circumīrēris" , "form_ipa" : "kirkumiːreːris" } ,
10115: { "form" : "circumīrēmur" , "form_ipa" : "kirkumiːreːmur" } ,
10115: { "form" : "circumīrēminī" , "form_ipa" : "kirkumiːreːminiː" } ,
10115: { "form" : "circumīrentur" , "form_ipa" : "kirkumiːrentur" } ,
10115: { "form" : "circumīre" , "form_ipa" : "kirkumiːre" } ,
10116: { "form" : "circumīminī" , "form_ipa" : "kirkumiːminiː" } ,
10116: { "form" : "circumītor" , "form_ipa" : "kirkumiːtor" } ,
10116: { "form" : "circumītor" , "form_ipa" : "kirkumiːtor" } ,
10116: { "form" : "circumeuntor" , "form_ipa" : "kirkumeuntor" } ,
10116: { "form" : "circumeundus" , "form_ipa" : "kirkumeundus" } ,
10117: { "form" : "circumeunda" , "form_ipa" : "kirkumeunda" } ,
10117: { "form" : "circumeundum" , "form_ipa" : "kirkumeundum" } ,
10117: { "form" : "circumeundī" , "form_ipa" : "kirkumeundiː" } ,
10117: { "form" : "circumeundae" , "form_ipa" : "kirkumeundaj" } ,
10117: { "form" : "circumeundī" , "form_ipa" : "kirkumeundiː" } ,
10117: { "form" : "circumeundō" , "form_ipa" : "kirkumeundoː" } ,
10117: { "form" : "circumeundae" , "form_ipa" : "kirkumeundaj" } ,
10117: { "form" : "circumeundō" , "form_ipa" : "kirkumeundoː" } ,
10117: { "form" : "circumeundum" , "form_ipa" : "kirkumeundum" } ,
10117: { "form" : "circumeundam" , "form_ipa" : "kirkumeundam" } ,
10118: { "form" : "circumeunde" , "form_ipa" : "kirkumeunde" } ,
10118: { "form" : "circumeunda" , "form_ipa" : "kirkumeunda" } ,
10118: { "form" : "circumeundum" , "form_ipa" : "kirkumeundum" } ,
10118: { "form" : "circumeundō" , "form_ipa" : "kirkumeundoː" } ,
10118: { "form" : "circumeundā" , "form_ipa" : "kirkumeundaː" } ,
10118: { "form" : "circumeundō" , "form_ipa" : "kirkumeundoː" } ,
10118: { "form" : "circumeundī" , "form_ipa" : "kirkumeundiː" } ,
10118: { "form" : "circumeundae" , "form_ipa" : "kirkumeundaj" } ,
10118: { "form" : "circumeunda" , "form_ipa" : "kirkumeunda" } ,
10119: { "form" : "circumeundōrum" , "form_ipa" : "kirkumeundoːrum" } ,
10119: { "form" : "circumeundārum" , "form_ipa" : "kirkumeundaːrum" } ,
10119: { "form" : "circumeundōrum" , "form_ipa" : "kirkumeundoːrum" } ,
10119: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
10119: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
10119: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
10119: { "form" : "circumeundōs" , "form_ipa" : "kirkumeundoːs" } ,
10119: { "form" : "circumeundās" , "form_ipa" : "kirkumeundaːs" } ,
10119: { "form" : "circumeunda" , "form_ipa" : "kirkumeunda" } ,
10119: { "form" : "circumeundī" , "form_ipa" : "kirkumeundiː" } ,
10120: { "form" : "circumeundae" , "form_ipa" : "kirkumeundaj" } ,
10120: { "form" : "circumeunda" , "form_ipa" : "kirkumeunda" } ,
10120: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
10120: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
10120: { "form" : "circumeundīs" , "form_ipa" : "kirkumeundiːs" } ,
63757: { "form" : "prajtereor" , "form_ipa" : "prajtereor" } ,
63757: { "form" : "prajterīris" , "form_ipa" : "prajteriːris" } ,
63758: { "form" : "prajterīmur" , "form_ipa" : "prajteriːmur" } ,
63758: { "form" : "prajterīminī" , "form_ipa" : "prajteriːminiː" } ,
63758: { "form" : "prajtereuntur" , "form_ipa" : "prajtereuntur" } ,
63758: { "form" : "prajterībar" , "form_ipa" : "prajteriːbar" } ,
63758: { "form" : "prajterībāris" , "form_ipa" : "prajteriːbaːris" } ,
63758: { "form" : "prajterībāmur" , "form_ipa" : "prajteriːbaːmur" } ,
63758: { "form" : "prajterībāminī" , "form_ipa" : "prajteriːbaːminiː" } ,
63758: { "form" : "prajterībantur" , "form_ipa" : "prajteriːbantur" } ,
63758: { "form" : "prajterībor" , "form_ipa" : "prajteriːbor" } ,
63759: { "form" : "prajterīberis" , "form_ipa" : "prajteriːberis" } ,
63759: { "form" : "prajterībimur" , "form_ipa" : "prajteriːbimur" } ,
63759: { "form" : "prajterībiminī" , "form_ipa" : "prajteriːbiminiː" } ,
63759: { "form" : "prajterībuntur" , "form_ipa" : "prajteriːbuntur" } ,
63759: { "form" : "prajterear" , "form_ipa" : "prajterear" } ,
63759: { "form" : "prajtereāris" , "form_ipa" : "prajtereaːris" } ,
63759: { "form" : "prajtereāmur" , "form_ipa" : "prajtereaːmur" } ,
63759: { "form" : "prajtereāminī" , "form_ipa" : "prajtereaːminiː" } ,
63760: { "form" : "prajtereantur" , "form_ipa" : "prajtereantur" } ,
63760: { "form" : "prajterīrer" , "form_ipa" : "prajteriːrer" } ,
63760: { "form" : "prajterīrēris" , "form_ipa" : "prajteriːreːris" } ,
63760: { "form" : "prajterīrēmur" , "form_ipa" : "prajteriːreːmur" } ,
63760: { "form" : "prajterīrēminī" , "form_ipa" : "prajteriːreːminiː" } ,
63760: { "form" : "prajterīrentur" , "form_ipa" : "prajteriːrentur" } ,
63760: { "form" : "prajterīre" , "form_ipa" : "prajteriːre" } ,
63760: { "form" : "prajterīminī" , "form_ipa" : "prajteriːminiː" } ,
63760: { "form" : "prajterītor" , "form_ipa" : "prajteriːtor" } ,
63761: { "form" : "prajterītor" , "form_ipa" : "prajteriːtor" } ,
63761: { "form" : "prajtereuntor" , "form_ipa" : "prajtereuntor" } ,
63761: { "form" : "prajtereundus" , "form_ipa" : "prajtereundus" } ,
63761: { "form" : "prajtereunda" , "form_ipa" : "prajtereunda" } ,
63761: { "form" : "prajtereundum" , "form_ipa" : "prajtereundum" } ,
63762: { "form" : "prajtereundī" , "form_ipa" : "prajtereundiː" } ,
63762: { "form" : "prajtereundae" , "form_ipa" : "prajtereundaj" } ,
63762: { "form" : "prajtereundī" , "form_ipa" : "prajtereundiː" } ,
63762: { "form" : "prajtereundō" , "form_ipa" : "prajtereundoː" } ,
63762: { "form" : "prajtereundae" , "form_ipa" : "prajtereundaj" } ,
63762: { "form" : "prajtereundō" , "form_ipa" : "prajtereundoː" } ,
63762: { "form" : "prajtereundum" , "form_ipa" : "prajtereundum" } ,
63762: { "form" : "prajtereundam" , "form_ipa" : "prajtereundam" } ,
63762: { "form" : "prajtereunde" , "form_ipa" : "prajtereunde" } ,
63763: { "form" : "prajtereunda" , "form_ipa" : "prajtereunda" } ,
63763: { "form" : "prajtereundum" , "form_ipa" : "prajtereundum" } ,
63763: { "form" : "prajtereundō" , "form_ipa" : "prajtereundoː" } ,
63763: { "form" : "prajtereundā" , "form_ipa" : "prajtereundaː" } ,
63763: { "form" : "prajtereundō" , "form_ipa" : "prajtereundoː" } ,
63763: { "form" : "prajtereundī" , "form_ipa" : "prajtereundiː" } ,
63763: { "form" : "prajtereundae" , "form_ipa" : "prajtereundaj" } ,
63763: { "form" : "prajtereunda" , "form_ipa" : "prajtereunda" } ,
63763: { "form" : "prajtereundōrum" , "form_ipa" : "prajtereundoːrum" } ,
63763: { "form" : "prajtereundārum" , "form_ipa" : "prajtereundaːrum" } ,
63764: { "form" : "prajtereundōrum" , "form_ipa" : "prajtereundoːrum" } ,
63764: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
63764: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
63764: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
63764: { "form" : "prajtereundōs" , "form_ipa" : "prajtereundoːs" } ,
63764: { "form" : "prajtereundās" , "form_ipa" : "prajtereundaːs" } ,
63764: { "form" : "prajtereunda" , "form_ipa" : "prajtereunda" } ,
63764: { "form" : "prajtereundī" , "form_ipa" : "prajtereundiː" } ,
63764: { "form" : "prajtereundae" , "form_ipa" : "prajtereundaj" } ,
63764: { "form" : "prajtereunda" , "form_ipa" : "prajtereunda" } ,
63765: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
63765: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
63765: { "form" : "prajtereundīs" , "form_ipa" : "prajtereundiːs" } ,
67237: { "form" : "queor" , "form_ipa" : "kweor" } ,
67237: { "form" : "quīris" , "form_ipa" : "kwiːris" } ,
67237: { "form" : "quīmur" , "form_ipa" : "kwiːmur" } ,
67237: { "form" : "quīminī" , "form_ipa" : "kwiːminiː" } ,
67238: { "form" : "queuntur" , "form_ipa" : "kweuntur" } ,
67238: { "form" : "quībar" , "form_ipa" : "kwiːbar" } ,
67238: { "form" : "quībāris" , "form_ipa" : "kwiːbaːris" } ,
67238: { "form" : "quībāmur" , "form_ipa" : "kwiːbaːmur" } ,
67238: { "form" : "quībāminī" , "form_ipa" : "kwiːbaːminiː" } ,
67238: { "form" : "quībantur" , "form_ipa" : "kwiːbantur" } ,
67238: { "form" : "quībor" , "form_ipa" : "kwiːbor" } ,
67238: { "form" : "quīberis" , "form_ipa" : "kwiːberis" } ,
67239: { "form" : "quībimur" , "form_ipa" : "kwiːbimur" } ,
67239: { "form" : "quībiminī" , "form_ipa" : "kwiːbiminiː" } ,
67239: { "form" : "quībuntur" , "form_ipa" : "kwiːbuntur" } ,
67239: { "form" : "quear" , "form_ipa" : "kwear" } ,
67239: { "form" : "queāris" , "form_ipa" : "kweaːris" } ,
67239: { "form" : "queāmur" , "form_ipa" : "kweaːmur" } ,
67239: { "form" : "queāminī" , "form_ipa" : "kweaːminiː" } ,
67239: { "form" : "queantur" , "form_ipa" : "kweantur" } ,
67239: { "form" : "quīrer" , "form_ipa" : "kwiːrer" } ,
67240: { "form" : "quīrēris" , "form_ipa" : "kwiːreːris" } ,
67240: { "form" : "quīrēmur" , "form_ipa" : "kwiːreːmur" } ,
67240: { "form" : "quīrēminī" , "form_ipa" : "kwiːreːminiː" } ,
67240: { "form" : "quīrentur" , "form_ipa" : "kwiːrentur" } ,
67240: { "form" : "quīre" , "form_ipa" : "kwiːre" } ,
67240: { "form" : "quīminī" , "form_ipa" : "kwiːminiː" } ,
67240: { "form" : "quītor" , "form_ipa" : "kwiːtor" } ,
67240: { "form" : "quītor" , "form_ipa" : "kwiːtor" } ,
67240: { "form" : "queuntor" , "form_ipa" : "kweuntor" } ,
67241: { "form" : "queundus" , "form_ipa" : "kweundus" } ,
67241: { "form" : "queunda" , "form_ipa" : "kweunda" } ,
67241: { "form" : "queundum" , "form_ipa" : "kweundum" } ,
67241: { "form" : "queundī" , "form_ipa" : "kweundiː" } ,
67241: { "form" : "queundae" , "form_ipa" : "kweundaj" } ,
67242: { "form" : "queundī" , "form_ipa" : "kweundiː" } ,
67242: { "form" : "queundō" , "form_ipa" : "kweundoː" } ,
67242: { "form" : "queundae" , "form_ipa" : "kweundaj" } ,
67242: { "form" : "queundō" , "form_ipa" : "kweundoː" } ,
67242: { "form" : "queundum" , "form_ipa" : "kweundum" } ,
67242: { "form" : "queundam" , "form_ipa" : "kweundam" } ,
67242: { "form" : "queunde" , "form_ipa" : "kweunde" } ,
67242: { "form" : "queunda" , "form_ipa" : "kweunda" } ,
67242: { "form" : "queundum" , "form_ipa" : "kweundum" } ,
67243: { "form" : "queundō" , "form_ipa" : "kweundoː" } ,
67243: { "form" : "queundā" , "form_ipa" : "kweundaː" } ,
67243: { "form" : "queundō" , "form_ipa" : "kweundoː" } ,
67243: { "form" : "queundī" , "form_ipa" : "kweundiː" } ,
67243: { "form" : "queundae" , "form_ipa" : "kweundaj" } ,
67243: { "form" : "queunda" , "form_ipa" : "kweunda" } ,
67243: { "form" : "queundōrum" , "form_ipa" : "kweundoːrum" } ,
67243: { "form" : "queundārum" , "form_ipa" : "kweundaːrum" } ,
67243: { "form" : "queundōrum" , "form_ipa" : "kweundoːrum" } ,
67243: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
67244: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
67244: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
67244: { "form" : "queundōs" , "form_ipa" : "kweundoːs" } ,
67244: { "form" : "queundās" , "form_ipa" : "kweundaːs" } ,
67244: { "form" : "queunda" , "form_ipa" : "kweunda" } ,
67244: { "form" : "queundī" , "form_ipa" : "kweundiː" } ,
67244: { "form" : "queundae" , "form_ipa" : "kweundaj" } ,
67244: { "form" : "queunda" , "form_ipa" : "kweunda" } ,
67244: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
67244: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
67245: { "form" : "queundīs" , "form_ipa" : "kweundiːs" } ,
76889: { "form" : "subeor" , "form_ipa" : "subeor" } ,
76889: { "form" : "subīris" , "form_ipa" : "subiːris" } ,
76889: { "form" : "subīmur" , "form_ipa" : "subiːmur" } ,
76889: { "form" : "subīminī" , "form_ipa" : "subiːminiː" } ,
76890: { "form" : "subeuntur" , "form_ipa" : "subeuntur" } ,
76890: { "form" : "subībar" , "form_ipa" : "subiːbar" } ,
76890: { "form" : "subībāris" , "form_ipa" : "subiːbaːris" } ,
76890: { "form" : "subībāmur" , "form_ipa" : "subiːbaːmur" } ,
76890: { "form" : "subībāminī" , "form_ipa" : "subiːbaːminiː" } ,
76890: { "form" : "subībantur" , "form_ipa" : "subiːbantur" } ,
76890: { "form" : "subībor" , "form_ipa" : "subiːbor" } ,
76890: { "form" : "subīberis" , "form_ipa" : "subiːberis" } ,
76891: { "form" : "subībimur" , "form_ipa" : "subiːbimur" } ,
76891: { "form" : "subībiminī" , "form_ipa" : "subiːbiminiː" } ,
76891: { "form" : "subībuntur" , "form_ipa" : "subiːbuntur" } ,
76891: { "form" : "subear" , "form_ipa" : "subear" } ,
76891: { "form" : "subeāris" , "form_ipa" : "subeaːris" } ,
76891: { "form" : "subeāmur" , "form_ipa" : "subeaːmur" } ,
76891: { "form" : "subeāminī" , "form_ipa" : "subeaːminiː" } ,
76891: { "form" : "subeantur" , "form_ipa" : "subeantur" } ,
76891: { "form" : "subīrer" , "form_ipa" : "subiːrer" } ,
76892: { "form" : "subīrēris" , "form_ipa" : "subiːreːris" } ,
76892: { "form" : "subīrēmur" , "form_ipa" : "subiːreːmur" } ,
76892: { "form" : "subīrēminī" , "form_ipa" : "subiːreːminiː" } ,
76892: { "form" : "subīrentur" , "form_ipa" : "subiːrentur" } ,
76892: { "form" : "subīre" , "form_ipa" : "subiːre" } ,
76892: { "form" : "subīminī" , "form_ipa" : "subiːminiː" } ,
76892: { "form" : "subītor" , "form_ipa" : "subiːtor" } ,
76892: { "form" : "subītor" , "form_ipa" : "subiːtor" } ,
76892: { "form" : "subeuntor" , "form_ipa" : "subeuntor" } ,
76893: { "form" : "subeundus" , "form_ipa" : "subeundus" } ,
76893: { "form" : "subeunda" , "form_ipa" : "subeunda" } ,
76893: { "form" : "subeundum" , "form_ipa" : "subeundum" } ,
76893: { "form" : "subeundī" , "form_ipa" : "subeundiː" } ,
76893: { "form" : "subeundae" , "form_ipa" : "subeundaj" } ,
76894: { "form" : "subeundī" , "form_ipa" : "subeundiː" } ,
76894: { "form" : "subeundō" , "form_ipa" : "subeundoː" } ,
76894: { "form" : "subeundae" , "form_ipa" : "subeundaj" } ,
76894: { "form" : "subeundō" , "form_ipa" : "subeundoː" } ,
76894: { "form" : "subeundum" , "form_ipa" : "subeundum" } ,
76894: { "form" : "subeundam" , "form_ipa" : "subeundam" } ,
76894: { "form" : "subeunde" , "form_ipa" : "subeunde" } ,
76894: { "form" : "subeunda" , "form_ipa" : "subeunda" } ,
76894: { "form" : "subeundum" , "form_ipa" : "subeundum" } ,
76895: { "form" : "subeundō" , "form_ipa" : "subeundoː" } ,
76895: { "form" : "subeundā" , "form_ipa" : "subeundaː" } ,
76895: { "form" : "subeundō" , "form_ipa" : "subeundoː" } ,
76895: { "form" : "subeundī" , "form_ipa" : "subeundiː" } ,
76895: { "form" : "subeundae" , "form_ipa" : "subeundaj" } ,
76895: { "form" : "subeunda" , "form_ipa" : "subeunda" } ,
76895: { "form" : "subeundōrum" , "form_ipa" : "subeundoːrum" } ,
76895: { "form" : "subeundārum" , "form_ipa" : "subeundaːrum" } ,
76895: { "form" : "subeundōrum" , "form_ipa" : "subeundoːrum" } ,
76895: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
76896: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
76896: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
76896: { "form" : "subeundōs" , "form_ipa" : "subeundoːs" } ,
76896: { "form" : "subeundās" , "form_ipa" : "subeundaːs" } ,
76896: { "form" : "subeunda" , "form_ipa" : "subeunda" } ,
76896: { "form" : "subeundī" , "form_ipa" : "subeundiː" } ,
76896: { "form" : "subeundae" , "form_ipa" : "subeundaj" } ,
76896: { "form" : "subeunda" , "form_ipa" : "subeunda" } ,
76896: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
76896: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
76897: { "form" : "subeundīs" , "form_ipa" : "subeundiːs" } ,
81410: { "form" : "trānseor" , "form_ipa" : "traːnseor" } ,
81410: { "form" : "trānsīris" , "form_ipa" : "traːnsiːris" } ,
81411: { "form" : "trānsīmur" , "form_ipa" : "traːnsiːmur" } ,
81411: { "form" : "trānsīminī" , "form_ipa" : "traːnsiːminiː" } ,
81411: { "form" : "trānseuntur" , "form_ipa" : "traːnseuntur" } ,
81411: { "form" : "trānsībar" , "form_ipa" : "traːnsiːbar" } ,
81411: { "form" : "trānsībāris" , "form_ipa" : "traːnsiːbaːris" } ,
81411: { "form" : "trānsībāmur" , "form_ipa" : "traːnsiːbaːmur" } ,
81411: { "form" : "trānsībāminī" , "form_ipa" : "traːnsiːbaːminiː" } ,
81411: { "form" : "trānsībantur" , "form_ipa" : "traːnsiːbantur" } ,
81411: { "form" : "trānsībor" , "form_ipa" : "traːnsiːbor" } ,
81412: { "form" : "trānsīberis" , "form_ipa" : "traːnsiːberis" } ,
81412: { "form" : "trānsībimur" , "form_ipa" : "traːnsiːbimur" } ,
81412: { "form" : "trānsībiminī" , "form_ipa" : "traːnsiːbiminiː" } ,
81412: { "form" : "trānsībuntur" , "form_ipa" : "traːnsiːbuntur" } ,
81412: { "form" : "trānsear" , "form_ipa" : "traːnsear" } ,
81412: { "form" : "trānseāris" , "form_ipa" : "traːnseaːris" } ,
81412: { "form" : "trānseāmur" , "form_ipa" : "traːnseaːmur" } ,
81412: { "form" : "trānseāminī" , "form_ipa" : "traːnseaːminiː" } ,
81413: { "form" : "trānseantur" , "form_ipa" : "traːnseantur" } ,
81413: { "form" : "trānsīrer" , "form_ipa" : "traːnsiːrer" } ,
81413: { "form" : "trānsīrēris" , "form_ipa" : "traːnsiːreːris" } ,
81413: { "form" : "trānsīrēmur" , "form_ipa" : "traːnsiːreːmur" } ,
81413: { "form" : "trānsīrēminī" , "form_ipa" : "traːnsiːreːminiː" } ,
81413: { "form" : "trānsīrentur" , "form_ipa" : "traːnsiːrentur" } ,
81413: { "form" : "trānsīre" , "form_ipa" : "traːnsiːre" } ,
81413: { "form" : "trānsīminī" , "form_ipa" : "traːnsiːminiː" } ,
81413: { "form" : "trānsītor" , "form_ipa" : "traːnsiːtor" } ,
81414: { "form" : "trānsītor" , "form_ipa" : "traːnsiːtor" } ,
81414: { "form" : "trānseuntor" , "form_ipa" : "traːnseuntor" } ,
81414: { "form" : "trānseundus" , "form_ipa" : "traːnseundus" } ,
81414: { "form" : "trānseunda" , "form_ipa" : "traːnseunda" } ,
81414: { "form" : "trānseundum" , "form_ipa" : "traːnseundum" } ,
81415: { "form" : "trānseundī" , "form_ipa" : "traːnseundiː" } ,
81415: { "form" : "trānseundae" , "form_ipa" : "traːnseundaj" } ,
81415: { "form" : "trānseundī" , "form_ipa" : "traːnseundiː" } ,
81415: { "form" : "trānseundō" , "form_ipa" : "traːnseundoː" } ,
81415: { "form" : "trānseundae" , "form_ipa" : "traːnseundaj" } ,
81415: { "form" : "trānseundō" , "form_ipa" : "traːnseundoː" } ,
81415: { "form" : "trānseundum" , "form_ipa" : "traːnseundum" } ,
81415: { "form" : "trānseundam" , "form_ipa" : "traːnseundam" } ,
81415: { "form" : "trānseunde" , "form_ipa" : "traːnseunde" } ,
81416: { "form" : "trānseunda" , "form_ipa" : "traːnseunda" } ,
81416: { "form" : "trānseundum" , "form_ipa" : "traːnseundum" } ,
81416: { "form" : "trānseundō" , "form_ipa" : "traːnseundoː" } ,
81416: { "form" : "trānseundā" , "form_ipa" : "traːnseundaː" } ,
81416: { "form" : "trānseundō" , "form_ipa" : "traːnseundoː" } ,
81416: { "form" : "trānseundī" , "form_ipa" : "traːnseundiː" } ,
81416: { "form" : "trānseundae" , "form_ipa" : "traːnseundaj" } ,
81416: { "form" : "trānseunda" , "form_ipa" : "traːnseunda" } ,
81416: { "form" : "trānseundōrum" , "form_ipa" : "traːnseundoːrum" } ,
81416: { "form" : "trānseundārum" , "form_ipa" : "traːnseundaːrum" } ,
81417: { "form" : "trānseundōrum" , "form_ipa" : "traːnseundoːrum" } ,
81417: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
81417: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
81417: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
81417: { "form" : "trānseundōs" , "form_ipa" : "traːnseundoːs" } ,
81417: { "form" : "trānseundās" , "form_ipa" : "traːnseundaːs" } ,
81417: { "form" : "trānseunda" , "form_ipa" : "traːnseunda" } ,
81417: { "form" : "trānseundī" , "form_ipa" : "traːnseundiː" } ,
81417: { "form" : "trānseundae" , "form_ipa" : "traːnseundaj" } ,
81417: { "form" : "trānseunda" , "form_ipa" : "traːnseunda" } ,
81418: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
81418: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
81418: { "form" : "trānseundīs" , "form_ipa" : "traːnseundiːs" } ,
83315: { "form" : "uēneor" , "form_ipa" : "weːneor" } ,
83315: { "form" : "uēnīris" , "form_ipa" : "weːniːris" } ,
83316: { "form" : "uēnīmur" , "form_ipa" : "weːniːmur" } ,
83316: { "form" : "uēnīminī" , "form_ipa" : "weːniːminiː" } ,
83316: { "form" : "uēneuntur" , "form_ipa" : "weːneuntur" } ,
83316: { "form" : "uēnībar" , "form_ipa" : "weːniːbar" } ,
83316: { "form" : "uēnībāris" , "form_ipa" : "weːniːbaːris" } ,
83316: { "form" : "uēnībāmur" , "form_ipa" : "weːniːbaːmur" } ,
83316: { "form" : "uēnībāminī" , "form_ipa" : "weːniːbaːminiː" } ,
83316: { "form" : "uēnībantur" , "form_ipa" : "weːniːbantur" } ,
83316: { "form" : "uēnībor" , "form_ipa" : "weːniːbor" } ,
83317: { "form" : "uēnīberis" , "form_ipa" : "weːniːberis" } ,
83317: { "form" : "uēnībimur" , "form_ipa" : "weːniːbimur" } ,
83317: { "form" : "uēnībiminī" , "form_ipa" : "weːniːbiminiː" } ,
83317: { "form" : "uēnībuntur" , "form_ipa" : "weːniːbuntur" } ,
83317: { "form" : "uēnear" , "form_ipa" : "weːnear" } ,
83317: { "form" : "uēneāris" , "form_ipa" : "weːneaːris" } ,
83317: { "form" : "uēneāmur" , "form_ipa" : "weːneaːmur" } ,
83317: { "form" : "uēneāminī" , "form_ipa" : "weːneaːminiː" } ,
83318: { "form" : "uēneantur" , "form_ipa" : "weːneantur" } ,
83318: { "form" : "uēnīrer" , "form_ipa" : "weːniːrer" } ,
83318: { "form" : "uēnīrēris" , "form_ipa" : "weːniːreːris" } ,
83318: { "form" : "uēnīrēmur" , "form_ipa" : "weːniːreːmur" } ,
83318: { "form" : "uēnīrēminī" , "form_ipa" : "weːniːreːminiː" } ,
83318: { "form" : "uēnīrentur" , "form_ipa" : "weːniːrentur" } ,
83318: { "form" : "uēnīre" , "form_ipa" : "weːniːre" } ,
83318: { "form" : "uēnīminī" , "form_ipa" : "weːniːminiː" } ,
83318: { "form" : "uēnītor" , "form_ipa" : "weːniːtor" } ,
83319: { "form" : "uēnītor" , "form_ipa" : "weːniːtor" } ,
83319: { "form" : "uēneuntor" , "form_ipa" : "weːneuntor" } ,
83319: { "form" : "uēneundus" , "form_ipa" : "weːneundus" } ,
83319: { "form" : "uēneunda" , "form_ipa" : "weːneunda" } ,
83319: { "form" : "uēneundum" , "form_ipa" : "weːneundum" } ,
83320: { "form" : "uēneundī" , "form_ipa" : "weːneundiː" } ,
83320: { "form" : "uēneundae" , "form_ipa" : "weːneundaj" } ,
83320: { "form" : "uēneundī" , "form_ipa" : "weːneundiː" } ,
83320: { "form" : "uēneundō" , "form_ipa" : "weːneundoː" } ,
83320: { "form" : "uēneundae" , "form_ipa" : "weːneundaj" } ,
83320: { "form" : "uēneundō" , "form_ipa" : "weːneundoː" } ,
83320: { "form" : "uēneundum" , "form_ipa" : "weːneundum" } ,
83320: { "form" : "uēneundam" , "form_ipa" : "weːneundam" } ,
83320: { "form" : "uēneunde" , "form_ipa" : "weːneunde" } ,
83321: { "form" : "uēneunda" , "form_ipa" : "weːneunda" } ,
83321: { "form" : "uēneundum" , "form_ipa" : "weːneundum" } ,
83321: { "form" : "uēneundō" , "form_ipa" : "weːneundoː" } ,
83321: { "form" : "uēneundā" , "form_ipa" : "weːneundaː" } ,
83321: { "form" : "uēneundō" , "form_ipa" : "weːneundoː" } ,
83321: { "form" : "uēneundī" , "form_ipa" : "weːneundiː" } ,
83321: { "form" : "uēneundae" , "form_ipa" : "weːneundaj" } ,
83321: { "form" : "uēneunda" , "form_ipa" : "weːneunda" } ,
83321: { "form" : "uēneundōrum" , "form_ipa" : "weːneundoːrum" } ,
83321: { "form" : "uēneundārum" , "form_ipa" : "weːneundaːrum" } ,
83322: { "form" : "uēneundōrum" , "form_ipa" : "weːneundoːrum" } ,
83322: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" } ,
83322: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" } ,
83322: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" } ,
83322: { "form" : "uēneundōs" , "form_ipa" : "weːneundoːs" } ,
83322: { "form" : "uēneundās" , "form_ipa" : "weːneundaːs" } ,
83322: { "form" : "uēneunda" , "form_ipa" : "weːneunda" } ,
83322: { "form" : "uēneundī" , "form_ipa" : "weːneundiː" } ,
83322: { "form" : "uēneundae" , "form_ipa" : "weːneundaj" } ,
83322: { "form" : "uēneunda" , "form_ipa" : "weːneunda" } ,
83323: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" } ,
83323: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" } ,
83323: { "form" : "uēneundīs" , "form_ipa" : "weːneundiːs" }
}

In [7]:
for row in changes:
    #print(row)
    for column in changes[row]:
        #print(column)
        #print(changes[row][column])
        LatInfLexi_verbs.loc[row,column] = changes[row][column]

C:\Users\matteo.pellegrini\AppData\Local\Temp\ipykernel_1424\2998859583.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'adeor' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  LatInfLexi_verbs.loc[row,column] = changes[row][column]


# Distinguishing defective cells from missing data

In [8]:
defective_verbs = pd.read_csv("./etc/verbs_defectiveness_mapping_manual.csv", sep="\t", index_col="lexeme")
defective_nouns = pd.read_csv("./etc/nouns_defectiveness_mapping_manual.csv", sep="\t", index_col="lexeme")

In [9]:
for i in LatInfLexi_nouns.index:
    if LatInfLexi_nouns.loc[i,"form"] == "#DEF#" and LatInfLexi_nouns.loc[i,"form_IPA"] == "#DEF#" :
        if defective_nouns.loc[LatInfLexi_nouns.loc[i,"lexeme"],"systematicDefectiveness"] != "plurale tantum":
            LatInfLexi_nouns.loc[i,"form"] = ""
            LatInfLexi_nouns.loc[i,"form_IPA"] = ""
            #print(LatInfLexi_nouns.loc[i,"lexeme"],LatInfLexi_nouns.loc[i,"PoSTag:features"],LatInfLexi_nouns.loc[i,"form_IPA"],type(LatInfLexi_nouns.loc[i,"form_IPA"]))

In [10]:
LatInfLexi_nouns = LatInfLexi_nouns[LatInfLexi_nouns["form"] != ""]

In [11]:
never_defective_cells = ['VERB:Sup+-+-+-+Act+-+-+Acc+-', 'VERB:Sup+-+-+-+Pass+-+-+Abl+-', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Nom+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Nom+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Nom+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Gen+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Gen+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Gen+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Dat+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Dat+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Dat+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Acc+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Acc+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Acc+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Voc+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Voc+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Voc+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Abl+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Abl+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Sing+Abl+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Nom+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Nom+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Nom+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Gen+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Gen+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Gen+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Dat+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Dat+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Dat+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Acc+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Acc+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Acc+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Voc+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Voc+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Voc+Neut', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Abl+Masc', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Abl+Fem', 'VERB:Part+-+Past+Perf+Pass+-+Plur+Abl+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Nom+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Nom+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Nom+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Gen+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Gen+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Gen+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Dat+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Dat+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Dat+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Acc+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Acc+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Acc+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Voc+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Voc+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Voc+Neut', 'VERB:Part+-+Fut+-+Act+-+Sing+Abl+Masc', 'VERB:Part+-+Fut+-+Act+-+Sing+Abl+Fem', 'VERB:Part+-+Fut+-+Act+-+Sing+Abl+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Nom+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Nom+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Nom+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Gen+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Gen+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Gen+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Dat+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Dat+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Dat+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Acc+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Acc+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Acc+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Voc+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Voc+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Voc+Neut', 'VERB:Part+-+Fut+-+Act+-+Plur+Abl+Masc', 'VERB:Part+-+Fut+-+Act+-+Plur+Abl+Fem', 'VERB:Part+-+Fut+-+Act+-+Plur+Abl+Neut']

In [12]:
for i in tqdm(LatInfLexi_verbs.index):
    if LatInfLexi_verbs.loc[i,"form"] == "#DEF#" and LatInfLexi_verbs.loc[i,"form_IPA"] == "#DEF#":
        if defective_verbs.loc[LatInfLexi_verbs.loc[i,"lexeme"],"systematicDefectiveness"] == "none" or LatInfLexi_verbs.loc[i,"PoSTag:features"] in never_defective_cells:
            LatInfLexi_verbs.loc[i,"form"] = ""
            LatInfLexi_verbs.loc[i,"form_IPA"] = ""
            #print(LatInfLexi_verbs.loc[i,"lexeme"],LatInfLexi_verbs.loc[i,"PoSTag:features"],LatInfLexi_verbs.loc[i,"form_IPA"],type(LatInfLexi_verbs.loc[i,"form_IPA"]))

100%|████████████████████████████████████████████████████████████████████████| 850392/850392 [10:13<00:00, 1387.02it/s]


In [13]:
LatInfLexi_verbs = LatInfLexi_verbs[LatInfLexi_verbs["form"] != ""]

# Combining forms tables

Adding POS

In [14]:
LatInfLexi_verbs["POS"] = "verb"
LatInfLexi_nouns["POS"] = "noun"

Generating form_id

In [15]:
def add_form_id(df, suffix=""):
    df.index.name = "form_id"
    df.reset_index(inplace=True)
    df["form_id"] = "form_" + df["form_id"].apply(str) + suffix

add_form_id(LatInfLexi_verbs, "_v")
add_form_id(LatInfLexi_nouns, "_n")

Concatenating forms tables:

In [16]:
LatInfLexi_forms = pd.concat([LatInfLexi_verbs, LatInfLexi_nouns])

# Converting forms to Paralex format

Obtaining columns in Paralex format:

In [17]:
col_map = {"form": "orth_form", "form_IPA": "phon_form", "PoSTag:features": "cell",
           "freqTFTL":"frequency",
           "freqAntiquitas":"frequency_Antiquitas" ,
           "freqAetasPatrum":"frequency_AetasPatrum",
            "freqMediumAeuum":"frequency_MediumAeuum",
            "freqRecentiorLatinitas":"frequency_RecentiorLatinitas"
           }
LatInfLexi_forms.rename(col_map, axis=1, inplace=True)

Mapping cells to new scheme:

In [18]:
LatInfLexi_cells_mapper = LatInfLexi_cells.set_index("LatInFlexi-cell").cell_id.to_dict()
LatInfLexi_forms.loc[:,"cell"] = LatInfLexi_forms.cell.map(LatInfLexi_cells_mapper)

In [19]:
LatInfLexi_forms.sample(5)

,form_id,lexeme,cell,orth_form,phon_form,frequency,frequency_Antiquitas,frequency_AetasPatrum,frequency_MediumAeuum,frequency_RecentiorLatinitas,form_ipa,POS
628084,form_660797_v,prosequor,prs.act.ptcp.voc.m.pl,prōsequentēs,proːsekwenteːs,69,3,9,50,7,NaN,verb
581431,form_611177_v,pollicitor,fut.pass.ind.2.pl,pollicitābiminī,pollikitaːbiminiː,0,0,0,0,0,NaN,verb
529371,form_555895_v,opto,prs.act.ptcp.voc.m.pl,optantēs,optanteːs,74,5,27,40,2,NaN,verb
64832,form_67727_v,aspergo,fprf.act.ind.3.sg,asperserit,asperserit,12,1,7,4,0,NaN,verb
160471,form_168494_v,consumo,gdv.abl.m.sg,cōnsūmendō,koːnsuːmendoː,44,0,14,29,1,NaN,verb


Setting form_id as index

In [20]:
LatInfLexi_forms = LatInfLexi_forms.set_index("form_id")

In [21]:
LatInfLexi_forms

,lexeme,cell,orth_form,phon_form,frequency,frequency_Antiquitas,frequency_AetasPatrum,frequency_MediumAeuum,frequency_RecentiorLatinitas,form_ipa,POS
form_id,,,,,,,,,,,
form_0_v,abalieno,prs.act.ind.1.sg,abaliēnō,abalieːnoː,0,0,0,0,0,NaN,verb
form_1_v,abalieno,prs.act.ind.2.sg,abaliēnās,abalieːnaːs,0,0,0,0,0,NaN,verb
form_2_v,abalieno,prs.act.ind.3.sg,abaliēnat,abalieːnat,5,2,2,1,0,NaN,verb
form_3_v,abalieno,prs.act.ind.1.pl,abaliēnāmus,abalieːnaːmus,0,0,0,0,0,NaN,verb
form_4_v,abalieno,prs.act.ind.2.pl,abaliēnātis,abalieːnaːtis,2,1,1,0,0,NaN,verb
...,...,...,...,...,...,...,...,...,...,...,...
form_12451_n,uxor,gen.pl,uxōrum,uksoːrum,265,15,124,122,4,NaN,noun
form_12452_n,uxor,dat.pl,uxōribus,uksoːribus,725,30,317,373,5,NaN,noun
form_12453_n,uxor,acc.pl,uxōrēs,uksoːreːs,2073,90,1064,903,16,NaN,noun


Adding stress

In [22]:
C = r"b|d|ɡ|m|n|l|r|z|p|pʰ|f|t|tʰ|s|k|kʰ|h"
V = r"waj|[jw][aeiouy]ː|[aeiouy]ː?|[jw][aeiouy]|[aeiouy][jw]"
segmenter = f"(?:{C}|({V}))*?"


def search_vowels(word):
    segmented = regex.fullmatch(segmenter, word)
    return segmented.spans(1)

def find_latin_stress(word):

    if word == "#DEF#" or word == "":
        return word

    def stress(idxs):
        i = idxs[0]
        if word[i] in 'wj':
            i += 1
        return word[:i] + "ˈ" + word[i:]

    indexes = search_vowels(word)

    # 2 syllables or less => stress first syllable
    if len(indexes) <= 2:
        return stress(indexes[0])

    *_, antepenult, penult, ultimate = indexes

    # If the penult has a long vowel it is stressed
    if word[slice(*penult)][-1] in {"ː", "j", "w"}:
        return stress(penult)

    # Single C after the penult => short penult => stress antepenult
    if (ultimate[0] - penult[1]) < 2:
        return stress(antepenult)

    # For the rest, it depends on the consonant sequence
    c_seq = word[penult[1]:ultimate[0]]
    cl = re.compile("^[bdɡpctd]ʰ?[rl]$")

    # C seq is a liquid cluster => short penult => stress antepenult
    if cl.match(c_seq):
        return stress(antepenult)

    # other C sequence => long penult => stress penult
    return stress(penult)


LatInfLexi_forms.loc[:, "phon_form"] = LatInfLexi_forms["phon_form"].apply(find_latin_stress)

Separating sounds with spaces

In [23]:
def splitter(series, split_pattern):
    series = series.str.split(pat=split_pattern, regex=True)
    return series.apply(lambda x: " ".join([char for char in x if char]))

sounds = ['b', 'd', 'ɡ', 'm', 'n', 'l', 'r', 'z', 'p', 'pʰ', 'f', 't', 'tʰ', 's', 'k', 'kʰ', 'h', 'j', 'w', 'a', 'aː', 'e', 'eː', 'i', 'iː', 'o', 'oː', 'u', 'uː', 'ˈa', 'ˈaː', 'ˈe', 'ˈeː', 'ˈi', 'ˈiː', 'ˈo', 'ˈoː', 'ˈu', 'ˈuː', 'ˈy', 'ˈyː']
split_pattern = "(" + "|".join(sorted(sounds, key=len, reverse=True)) + ")"
LatInfLexi_forms["phon_form"] = splitter(LatInfLexi_forms["phon_form"], split_pattern)

# Adjusting the transcription

In [24]:
# Adding frequencies to the cells

In [25]:
cells_freq = LatInfLexi_forms.groupby("cell")[["frequency", 
                                              "frequency_Antiquitas",
                                              "frequency_AetasPatrum",
                                              "frequency_MediumAeuum",
                                              "frequency_RecentiorLatinitas"]].agg(sum)
cells_freq.index.name = "cell_id"

C:\Users\matteo.pellegrini\AppData\Local\Temp\ipykernel_1424\418782974.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  "frequency_RecentiorLatinitas"]].agg(sum)


In [26]:
cells_freq

,frequency,frequency_Antiquitas,frequency_AetasPatrum,frequency_MediumAeuum,frequency_RecentiorLatinitas
cell_id,,,,,
abl.pl,835283,115180,310927,382000,27176
abl.sg,2816933,297275,962861,1463839,92958
acc.pl,1565479,216650,557840,734860,56129
acc.sg,2663992,246582,945900,1391128,80382
dat.pl,835283,115180,310927,382000,27176
...,...,...,...,...,...
prs.pass.sbjv.3.sg,168334,9945,66246,83631,8512
sup.abl,156341,17168,41767,88748,8658
sup.acc,591906,50304,198546,323267,19789


In [27]:
LatInfLexi_cells.set_index("cell_id", inplace=True)

In [28]:
LatInfLexi_cells = pd.merge(LatInfLexi_cells, cells_freq, left_index=True, right_index=True)

# Creating the lexemes table

In [29]:
LatInfLexi_lexemes = LatInfLexi_forms.groupby(["lexeme", "POS"])[["frequency", 
                                                                      "frequency_Antiquitas",
                                                                      "frequency_AetasPatrum",
                                                                      "frequency_MediumAeuum",
                                                                      "frequency_RecentiorLatinitas"]].agg(sum).reset_index("POS", drop=False)
LatInfLexi_lexemes.index.name = "lexeme_id"

C:\Users\matteo.pellegrini\AppData\Local\Temp\ipykernel_1424\2816140279.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  "frequency_RecentiorLatinitas"]].agg(sum).reset_index("POS", drop=False)


In [30]:
LatInfLexi_lexemes

,POS,frequency,frequency_Antiquitas,frequency_AetasPatrum,frequency_MediumAeuum,frequency_RecentiorLatinitas
lexeme_id,,,,,,
abalieno,verb,393,101,176,113,3
abdico,verb,2256,773,760,683,40
abdo,verb,4850,971,2111,1644,124
abduco,verb,3194,937,1057,1116,84
abeo,verb,6626,1723,1466,3229,208
...,...,...,...,...,...,...
uulgus,noun,7917,1588,2524,2311,1494
uulnero,verb,10975,1167,4903,4784,121
uulnus,noun,18041,3618,7531,6637,255


# Output

Writing it all to file

In [31]:
LatInfLexi_lexemes.to_csv("LatInfLexi-lexemes.csv")
LatInfLexi_forms.to_csv("LatInfLexi-forms.csv")
LatInfLexi_cells.to_csv("LatInfLexi-cells.csv")

Remove temporary files from v1.1

In [32]:
%%bash
rm LatInfLexi-nouns.csv
rm LatInfLexi-verbs.csv
